In [ ]:
# データのダウンロード
!mkdir dataset
!wget "https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S" -O ./dataset/train.csv
!wget "https://drive.google.com/uc?export=download&id=1Mdyv3DPcb1AncxE7H739QWyPkMqcdjqI" -O ./dataset/dev.csv
!wget "https://drive.google.com/uc?export=download&id=1yHKfDrs6ZAx4fizToTF7GL7ROvGk7Ry5" -O ./dataset/test.csv

mkdir: cannot create directory ‘dataset’: File exists
--2021-01-09 07:46:41--  https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S
Resolving drive.google.com (drive.google.com)... 74.125.20.100, 74.125.20.101, 74.125.20.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.20.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kdsruqrt2ronbvuokp472bb74moof9ft/1610178375000/12400649359575842542/*/1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S?e=download [following]
--2021-01-09 07:46:41--  https://doc-0s-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kdsruqrt2ronbvuokp472bb74moof9ft/1610178375000/12400649359575842542/*/1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S?e=download
Resolving doc-0s-08-docs.googleusercontent.com (doc-0s-08-docs.googleusercontent.com)... 74.125.28.132, 2607:f8b0:400e:c04::84
Connectin

In [ ]:
import csv
import random

In [ ]:
# データの読み込み
train = list(csv.reader(open("./dataset/train.csv")))[1:] # 訓練データ．1行目はヘッダー行なので除外
dev = list(csv.reader(open("./dataset/dev.csv")))[1:] # 検証用データ．1行目はヘッダー行なので除外
test = list(csv.reader(open("./dataset/test.csv")))[1:] # 最終評価用データ．1行目はヘッダー行なので除外

# 訓練データのみシャッフル
random.seed(42)
random.shuffle(train)

train_movienames, train_titles, train_texts, train_ratings = zip(*train) # 訓練データ．
dev_movienames, dev_titles, dev_texts, dev_ratings = zip(*dev) # 検証用データ
test_movienames, test_titles, test_texts = zip(*test) # 最終評価用データ．rating は付与されていない
y_train = [int(rating) for rating in train_ratings] # rating が文字列で付与されているので，int 型に変換
y_dev = [int(rating) for rating in dev_ratings] # rating が文字列で付与されているので，int 型に変換

In [ ]:
# ツールのインストール
!pip install tensorflow_text

In [ ]:
'''
-----------------------------------------------------
Universal Sentence Encoder
title のみを用いて rating を予測する単純な分類器を作成
-----------------------------------------------------
'''

import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

X_train = use(train_titles)
X_dev = use(dev_titles)
X_test = use(test_titles)

# print(len(X_train), len(X_train[0]), X_train[0])

In [ ]:
'''
-----------------------------------------------------
分類器の訓練
-----------------------------------------------------
'''

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 分類器の訓練

best_c, best_score = 0, 0
for c in [0.1, 1, 10]:
    classifier = LogisticRegression(C=c, max_iter=1000)
    classifier.fit(X_train, y_train)
    dev_acc = accuracy_score(y_dev, classifier.predict(X_dev))
    if best_score < dev_acc:
        best_score = dev_acc
        best_c = c
    print("Dev accuracy = %1.3f\tC = %s" % (dev_acc, str(c)))
print("Best parameter: C = %s" % str(best_c))

Dev accuracy = 0.370	C = 0.1
Dev accuracy = 0.373	C = 1
Dev accuracy = 0.372	C = 10
Best parameter: C = 1


In [ ]:
'''
-----------------------------------------------------
評価
-----------------------------------------------------
'''

classifier = LogisticRegression(C=best_c, max_iter=1000)
classifier.fit(X_train, y_train)
y_test_pred = classifier.predict(X_test)

In [ ]:
result = "\n".join([str(rating) for rating in y_test_pred])

In [ ]:
open("./dataset/test_pred.txt", "w").write(result)

1999